https://satijalab.org/seurat/archive/v3.1/atacseq_integration_vignette.html![image.png](attachment:image.png)

https://satijalab.org/seurat/archive/v3.0/atacseq_integration_vignette.html

In [5]:
library(Seurat)
library(ggplot2)
library(patchwork)
data <- readRDS("../../data/mouse-skin/data_list.rds")
peaks <- data$Peaks
genes = data[['Gene Expression']]

`gencode.vM23.primary_assembly.annotation.gtf` http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M23/gencode.vM23.primary_assembly.annotation.gtf.gz

source: [10x Mouse reference, mm10 (GENCODE vM23/Ensembl 98)](https://support.10xgenomics.com/single-cell-gene-expression/software/release-notes/build#mm10_2020A)

In [3]:
# create a gene activity matrix from the peak matrix and GTF, using chromosomes 1:22, X, and Y.
# Peaks that fall within gene bodies, or 2kb upstream of a gene, are considered
activity.matrix <- CreateGeneActivityMatrix(peak.matrix = peaks, 
                                            annotation.file = "/data/pinello/PROJECTS/2019_08_RNA_Acceleration/chen_nbt_2019/analysis_HC/Mus_musculus.GRCm38.97.gtf", 
    seq.levels = c(1:19, "X", "Y"), upstream = 2000, verbose = TRUE)

Warning message:
“CreateGeneActivityMatrix functionality is being moved to Signac. Equivalent functionality can be achieved via the Signac::GeneActivity function; for more information on Signac, please see https://github.com/timoast/Signac”


In [2]:
atac <- readRDS("atac.rds")

In [3]:
str(atac)

Formal class 'Seurat' [package "Seurat"] with 13 slots
  ..@ assays      :List of 2
  .. ..$ ATAC    :Formal class 'Assay' [package "Seurat"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:139669728] 23 34 42 53 55 62 94 103 114 126 ...
  .. .. .. .. .. ..@ p       : int [1:34775] 0 1476 6204 10972 19687 22071 31448 37785 42524 57558 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 344592 34774
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:344592] "chrX:143482906-143483206" "chr6:3200976-3201276" "chr9:123461850-123462150" "chr1:56782095-56782395" ...
  .. .. .. .. .. .. ..$ : chr [1:34774] "R1.01.R2.01.R3.06.P1.55" "R1.01.R2.03.R3.68.P1.55" "R1.01.R2.05.R3.15.P1.53" "R1.01.R2.05.R3.40.P1.55" ...
  .. .. .. .. .. ..@ x       : num [1:139669728] 1 2 2 1 1 1 1 1 1 1 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix

In [4]:
activity.matrix <- atac@assays$ACTIVITY@counts

In [6]:
rna_count = data[["Gene Expression"]]
shared_genes = intersect(rownames(rna_count), rownames(activity.matrix))
shared_cells = intersect(colnames(rna_count), colnames(activity.matrix))
rna_count <- rna_count[,shared_cells]
activity.matrix <- activity.matrix[,shared_cells]
colnames(rna_count) = paste0("rna.", colnames(rna_count))
colnames(activity.matrix) = paste0("atac.", colnames(activity.matrix))

In [7]:
cat_count_matrix = cbind(rna_count[shared_genes,], activity.matrix[shared_genes,])

In [10]:
#vars = apply(cat_count_matrix, 1, var)
which(vars == 0)

named integer(0)

In [13]:
sum(vars==0)

[1] 0

In [9]:
str(cat_count_matrix)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:89421510] 105 902 912 1048 1078 1119 1174 1185 1206 1267 ...
  ..@ p       : int [1:69549] 0 312 742 1515 2279 2735 3869 4164 4852 6468 ...
  ..@ Dim     : int [1:2] 16743 69548
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:16743] "0610009B22Rik" "0610009L18Rik" "0610010F05Rik" "0610025J13Rik" ...
  .. ..$ : chr [1:69548] "rna.R1.01.R2.01.R3.06.P1.55" "rna.R1.01.R2.03.R3.68.P1.55" "rna.R1.01.R2.05.R3.15.P1.53" "rna.R1.01.R2.05.R3.40.P1.55" ...
  ..@ x       : num [1:89421510] 1 1 1 1 1 1 1 1 1 1 ...
  ..@ factors : list()


In [14]:
cat_mat.pca = prcomp(t(as.matrix(cat_count_matrix)), center = T, scale. = T, rank. = 20)

## Incorporate cell type

In [15]:
meta <- read.table("../../data/mouse-skin/GSM4156597_skin_celltype.txt", stringsAsFactors = F, head = T)

In [16]:
meta.ft = c(meta$celltype, meta$celltype)
names(meta.ft) = c(paste0("rna.", meta$rna.bc), paste0("atac.", meta$rna.bc))

In [18]:
str(cat_mat.pca)

List of 5
 $ sdev    : num [1:16743] 41 12.54 7.62 6.71 5.09 ...
 $ rotation: num [1:16743, 1:20] 0.00954 0.00338 0.00911 0.00622 0.00655 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:16743] "0610009B22Rik" "0610009L18Rik" "0610010F05Rik" "0610025J13Rik" ...
  .. ..$ : chr [1:20] "PC1" "PC2" "PC3" "PC4" ...
 $ center  : Named num [1:16743] 0.0915 0.0116 0.1238 0.0356 0.0716 ...
  ..- attr(*, "names")= chr [1:16743] "0610009B22Rik" "0610009L18Rik" "0610010F05Rik" "0610025J13Rik" ...
 $ scale   : Named num [1:16743] 0.354 0.11 0.416 0.223 0.325 ...
  ..- attr(*, "names")= chr [1:16743] "0610009B22Rik" "0610009L18Rik" "0610010F05Rik" "0610025J13Rik" ...
 $ x       : num [1:69548, 1:20] -27.6 -25.9 -19.7 -20 -25.4 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:69548] "rna.R1.01.R2.01.R3.06.P1.55" "rna.R1.01.R2.03.R3.68.P1.55" "rna.R1.01.R2.05.R3.15.P1.53" "rna.R1.01.R2.05.R3.40.P1.55" ...
  .. ..$ : chr [1:20] "PC1" "PC2" "PC3" "PC4" ...
 - attr(*, "class")= chr "prc

In [19]:
embedding = as.data.frame(cat_mat.pca$x[,1:20])
stopifnot(rownames(embedding) != embedding$bc)
embedding$cell_type = factor(meta.ft[rownames(embedding)])

In [20]:
embedding$batch = as.numeric(factor(sapply(strsplit(rownames(embedding), split = ".", fixed = T), function(s) paste0(s[2:length(s)], collapse = "."))))

In [21]:
str(embedding)

'data.frame':	69548 obs. of  22 variables:
 $ PC1      : num  -27.6 -25.9 -19.7 -20 -25.4 ...
 $ PC2      : num  2.386 -0.933 -10.604 -9.345 -1.75 ...
 $ PC3      : num  3.41 3.38 4.51 5.16 4.17 ...
 $ PC4      : num  -3.94 -5.44 -9.14 -10.12 -6.34 ...
 $ PC5      : num  -4.94 -4.93 -10.57 -14.5 -5.65 ...
 $ PC6      : num  2.67 1.96 3.82 2.81 1.59 ...
 $ PC7      : num  2.22 1.82 4.91 3.62 3.43 ...
 $ PC8      : num  -0.689 -0.389 -0.134 -1.307 -0.479 ...
 $ PC9      : num  -3.32 1.91 -8.47 11.94 -3.22 ...
 $ PC10     : num  4.36 1.91 5.34 2.55 3.75 ...
 $ PC11     : num  -0.636 -1.781 -4.075 -3.814 -1.372 ...
 $ PC12     : num  -1.82 -1.83 -3.93 -6.28 -3.13 ...
 $ PC13     : num  0.19919 0.00944 0.51446 0.41892 0.07957 ...
 $ PC14     : num  0.3451 -0.4878 0.2739 -0.0867 -0.5559 ...
 $ PC15     : num  -0.0886 -0.3191 0.281 -0.3893 -0.8238 ...
 $ PC16     : num  0.74337 -0.36865 -0.19395 -0.00269 1.05833 ...
 $ PC17     : num  -1.0641 0.3398 -0.6925 0.0297 -0.8814 ...
 $ PC18     : nu

In [22]:
write.table(embedding, "../../Raw/mouse-skin/integrated_embedding.txt", quote = F, sep = "\t")